# NLP con librería NLTK

En la instalación de **NLTK** no se incluyen los recursos necesarios para trabajar en PLN (las reglas de puntuación o las stopwords).    
Por tanto, la primera vez que se ejecuten las funciones de librería se solicitará que se descarguen estos recursos.    
Esto es algo que se puede hacer simplemente indicando a la función ```download()``` de *NLTK* los recursos requeridos.    

## Análisis de sentimientos en inglés con NLTK.   

Para realizar análisis de sentimientos con NLTK es necesario importar lo siguiente:

In [3]:
# Importamos librerías
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


### 1. Preprocesamiento de datos

Antes de poder realizar el análisis de sentimientos con NLTK, es necesario preprocesar los mensajes de texto para normalizar. 

Los pasos a llevar a cabo son:

- **Tokenización**: dividir el texto en palabras o frases más pequeñas llamadas tokens.
- **Eliminación de signos de puntuación y caracteres especiales**.
- **Conversión de texto a minúsculas** para normalizar el texto.
- **Eliminación de las stopwords** o palabras irrelevantes para el mensaje tales como “a”, “el”, “y”, etc.
- **Reducción de las palabras a su forma base (lemas)**.   

Estos pasos se pueden implementar con funciones de NLTK, tal como se muestra en el siguiente ejemplo:

In [5]:
# Procesado de texto
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

text = "I love the content on the Artificial Intelligence blog, articles are fantastic."

# Tokenización
tokens = word_tokenize(text)

# Eliminación de signos de puntuación
tokens = [token for token in tokens if token not in string.punctuation]

# Conversión a minúsculas
tokens = [token.lower() for token in tokens]

# Eliminación de stopwords
stop_words = set(stopwords.words('english'))
tokens = [token for token in tokens if token not in stop_words]

# Lematización
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(token) for token in tokens]

# Reconstrucción del texto preprocesado
preprocessed_text = ' '.join(tokens)
preprocessed_text

'love content artificial intelligence blog article fantastic'

Proceso ejecutado:    

- En primer lugar, se debe tokenizar las frase mediante la función ```word_tokenize()```, lo que divide está en una lista de palabras y signos de puntuación.    
- Posteriormente, mediante con herramientas estándar de Python, se eliminan los tokens que estén en la lista de signos de puntuación (```string.punctuation```) y se convierten los todo el texto a minúsculas.    
- Una vez homogeneizado el texto, se eliminan las stopwords que incluye NLTK.    
- A la hora de importar las stopwords es necesario indicar el idioma con el que se está trabajando ya que estas son diferentes. 
- Finalmente se lematiza los tokens para eliminar plurales y derivaciones.

### 2. Extracción de características   

Una vez preprocesado el texto, es necesario extraer las características de este antes de poder entrenar un modelo.    
Lo más habitual es emplear la frecuencia de las palabras como características. En NLTK esto se implementa mediante la clase **FreqDist** y se muestra en el siguiente código:

In [6]:
from nltk import FreqDist

features = {}
words = word_tokenize(preprocessed_text)
word_freq = FreqDist(words)

for word, freq in word_freq.items():
    features[word] = freq

features

{'love': 1,
 'content': 1,
 'artificial': 1,
 'intelligence': 1,
 'blog': 1,
 'article': 1,
 'fantastic': 1}

Obtenemos como resultado un diccionario con la palabra clave y el valor es el número de ocurrencias de cada una de estas.

## 3. Conjunto de datos de entrenamiento   

Para entrenar un modelo es necesario contar con un conjunto de datos de entrenamiento.   
A tal efecto, se crea una lista de tuplas con el mensaje y la etiqueta que se desea que le corresponda para el entrenamiento.    

A modo de ejemplo se puede probar con un listado de siete mensajes similar al siguiente:

In [8]:
training_data = [
    ("I love the content on the Artificial Intelligence blog, articles are fantastic.", "positive"),
    ("The code does not work, it gave me an error when executing it.", "negative"),
    ("I love this product!", "positive"),
    ("This movie was terrible.", "negative"),
    ("The weather is nice today.", "positive"),
    ("I feel so sad about the news.", "negative"),
    ("It's just an average book.", "neutral"),
    ("I don´t like milk.", "negative")
]

### Factorización del código

Se puede factorizar el código anterior para facilitar su uso a la hora de entrenar un modelo y que, globalmente, resulte más modular.    

Para ello se pueden crear dos funciones:    
- ```preprocess_text()``` para el preprocesado de texto y 
- ```extract_features()``` para la extracción de características.

In [9]:
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

def preprocess_text(text):
    """
    Realiza el preprocesamiento básico de un texto en inglés utilizando NLTK.

    Args:
        text (str): El texto a ser preprocesado.

    Returns:
        str: El texto preprocesado.
    """
    # Tokenización
    tokens = word_tokenize(text)

    # Eliminación de signos de puntuación
    tokens = [token for token in tokens if token not in string.punctuation]

    # Conversión a minúsculas
    tokens = [token.lower() for token in tokens]

    # Eliminación de stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Reconstrucción del texto preprocesado
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


def extract_features(text):
    """
    Extrae las características del texto utilizando NLTK y devuelve un diccionario de características.

    Args:
        text (str): El texto del cual extraer características.

    Returns:
        dict: Un diccionario que representa las características extraídas del texto.
    """
    features = {}
    words = word_tokenize(text)
    word_freq = FreqDist(words)

    for word, freq in word_freq.items():
        features[word] = freq

    return features

## 4. Entrenamiento del modelo.   

El análisis de sentimientos con NLTK se puede realizar usando un clasificador basado en Naive Bayes.    
NLTK proporciona una clase en la que se implementa este tipo de clasificadores.     

Empleando esta clase y las funciones creadas en la sección anterior se puede entrenar un modelo con los datos de ejemplo, tal como se muestra a continuación:

In [10]:
from nltk.classify import NaiveBayesClassifier

# Preprocesamiento de los datos de entrenamiento
preprocessed_training_data = [(preprocess_text(text), label) for text, label in training_data]

# Extracción de características de los datos de entrenamiento
training_features = [(extract_features(text), label) for text, label in preprocessed_training_data]

# Entrenamiento del clasificador Naive Bayes
classifier = NaiveBayesClassifier.train(training_features)

## 5. Clasificación de nuevos textos.

Una vez que el modelo está entrando, este se puede usar para clasificar nuevos textos.    
Simplemente es necesario preprocesar y extraer las características de la nueva cadena de texto para realizar la predicción con el clasificador.

In [11]:
# Nuevo texto para clasificar
# new_text = "I really enjoy the concert" # positivo
# new_text = "The concert was terrible"     # negativo
new_text = "I really love the concert"    # negativo

# Preprocesamiento del nuevo texto
preprocessed_text = preprocess_text(new_text)

# Extracción de características del nuevo texto
features = extract_features(preprocessed_text)

# Clasificación del nuevo texto
sentiment = classifier.classify(features)
print("Sentiment:", sentiment)

Sentiment: positive


## Análisis de sentimientos en español  

En este noteboook se ha explicado cómo hacer análisis de sentimiento en inglés. Si se usa el ejemplo para trabajar con texto en español, u otros idiomas, el resultado no será satisfactorio dado que se ha usado el listado de stopwords del inglés y un lematizador (*WordNetLemmatizer*) que no es adecuado para el español.

Por eso, para realizar análisis de sentimientos en español se dispone de otro notebook que usa la librería **spaCy**, la cúal dispone de las herramientas adecuadas para llevar a cabo correctamente esta tarea.   

##  Conclusiones.
NLTK es la librería de referencia para el procesado del lenguaje natural (PLN). Se trata de una librería que facilita el trabajo cuando se desea realizar análisis de sentimientos.    
Aunque, como ya hemos dicho, funciones clave como la lematización solamente funcionan en inglés, el uso de NLTK facilita comprender los pasos necesarios para realizar este tipo de análisis.